![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_Whisper.ipynb)

# Import OpenVino Whisper models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. So please make sure you have upgraded to the latest Spark NLP release.
- The Whisper model was introduced in `Spark NLP 5.1.0 and requires Spark version 3.4.1 and up.`
- Official models are supported, but not all custom models may work.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `openvino` extension and it's dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.31.0`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.31.0 optimum-intel openvino==2024.1 sentencepiece onnx==1.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 806.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.7/171.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 193.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13

- HuggingFace has an extension called Optimum which offers specialized model inference, including OpenVINO. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- We'll use the [whisper-tiny](https://huggingface.co/openai/whisper-tiny) model from HuggingFace as an example and export it with the `optimum-cli`.

In [2]:
MODEL_NAME = "openai/whisper-tiny"
EXPORT_PATH = f"export_openvino/{MODEL_NAME}"

In [3]:
! optimum-cli export openvino --model {MODEL_NAME} {EXPORT_PATH}

2024-08-22 16:14:23.019613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 16:14:23.385738: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 16:14:23.486503: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 16:14:26.081949: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Framework not specified. Using pt to export the model.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resum

We have to move additional model assets into a seperate folder, so that Spark NLP can load it properly.

In [4]:
! mkdir -p {EXPORT_PATH}/assets
! mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/*.json {EXPORT_PATH}/*.txt

Let's have a look inside these two directories and see what we are dealing with:

In [5]:
!ls -l {EXPORT_PATH}

total 259100
drwxr-xr-x 2 root root      4096 Aug 22 16:15 assets
-rw-r--r-- 1 root root 118209104 Aug 22 16:15 openvino_decoder_model.bin
-rw-r--r-- 1 root root    329053 Aug 22 16:15 openvino_decoder_model.xml
-rw-r--r-- 1 root root 113484384 Aug 22 16:15 openvino_decoder_with_past_model.bin
-rw-r--r-- 1 root root    274757 Aug 22 16:15 openvino_decoder_with_past_model.xml
-rw-r--r-- 1 root root  32833640 Aug 22 16:14 openvino_encoder_model.bin
-rw-r--r-- 1 root root    164142 Aug 22 16:14 openvino_encoder_model.xml


In [6]:
!ls -l {EXPORT_PATH}/assets

total 4308
-rw-r--r-- 1 root root   34604 Aug 22 16:14 added_tokens.json
-rw-r--r-- 1 root root    2243 Aug 22 16:14 config.json
-rw-r--r-- 1 root root    3742 Aug 22 16:14 generation_config.json
-rw-r--r-- 1 root root  493869 Aug 22 16:14 merges.txt
-rw-r--r-- 1 root root   52666 Aug 22 16:14 normalizer.json
-rw-r--r-- 1 root root     339 Aug 22 16:14 preprocessor_config.json
-rw-r--r-- 1 root root    2194 Aug 22 16:14 special_tokens_map.json
-rw-r--r-- 1 root root  283277 Aug 22 16:14 tokenizer_config.json
-rw-r--r-- 1 root root 2480466 Aug 22 16:14 tokenizer.json
-rw-r--r-- 1 root root 1036584 Aug 22 16:14 vocab.json


## Import and Save Whisper in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script
- Additionally, we need to upgrade Spark to version 3.4.1.

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash
! pip install -U pyspark==3.4.1

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 23.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=35520bb723dd6a52ac228a8c249191033e27475dc70be0af064dde9b1b780d3c
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Su

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `WhisperForCTC` which allows us to load the OpenVINO model
- Most params will be set automatically. They can also be set later after loading the model in `WhisperForCTC` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [ ]:
from sparknlp.annotator import *

# All these params should be identical to the original OpenVino model
whisper = (
    WhisperForCTC.loadSavedModel(f"{EXPORT_PATH}", spark)
    .setInputCols("audio_assembler")
    .setOutputCol("text")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
whisper.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your OpenVINO Whisper model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 414404
-rw-r--r-- 1 root root 198092144 Apr 12 10:38 decoder_model
-rw-r--r-- 1 root root 193333200 Apr 12 10:38 decoder_with_past_model
-rw-r--r-- 1 root root  32910123 Apr 12 10:38 encoder_model
drwxr-xr-x 6 root root      4096 Apr 12 10:38 fields
drwxr-xr-x 2 root root      4096 Apr 12 10:38 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny Whisper model 😊

In [ ]:
! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt

--2024-04-12 10:39:27--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/audio/txt/librispeech_asr_0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2199992 (2.1M) [text/plain]
Saving to: ‘librispeech_asr_0.txt’

librispeech_asr_0.t 100%[===================>]   2.10M  --.-KB/s    in 0.01s   

2024-04-12 10:39:27 (143 MB/s) - ‘librispeech_asr_0.txt’ saved [2199992/2199992]



In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

audioAssembler = AudioAssembler() \
    .setInputCol("audio_content") \
    .setOutputCol("audio_assembler")

speechToText = WhisperForCTC.load(f"{MODEL_NAME}_spark_nlp")

pipeline = Pipeline().setStages([audioAssembler, speechToText])

audio_path = "librispeech_asr_0.txt"
with open(audio_path) as file:
    raw_floats = [float(data) for data in file.read().strip().split("\n")]

processedAudioFloats = spark.createDataFrame([[raw_floats]]).toDF("audio_content")

result = pipeline.fit(processedAudioFloats).transform(processedAudioFloats)
result.select("text.result").show(truncate = False)

+------------------------------------------------------------------------------------------+
|result                                                                                    |
+------------------------------------------------------------------------------------------+
|[ Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.]|
+------------------------------------------------------------------------------------------+



That's it! You can now go wild and use hundreds of Whisper models from HuggingFace 🤗 in Spark NLP 🚀
